# <font color='Green'> Desercion UNaB </font>

### EDA de datos de las primeras 5 semanas de cursada de Analisis Matematico I (1er Cuatrimestre 2021)

In [7]:
import pandas as pd
import os

# directorio donde están los datos. Cambiar según corresponda.
os.chdir('C:/Users/Gonza/Documents/codigo/desercion-UNaB/')

data = pd.read_csv("./datos_desercion_universitaria_5_semanas.csv")
data.head()

,Nombre.completo.del.usuario,Algún contenido ha sido publicado.,Comentario creado,Curso visto,Entrega creada.,Finalización de actividad de curso actualizada,Formulario de entrega visto.,Ha comenzado el intento,Intento de cuestionario visualizado,Intento del cuestionario revisado,...,Informe usuario del curso visto,Suscripción activada,Curso actualizado,Grupo creado,Elemento de calificación creado,Curso creado,Instancia de inscripción creada,Sección de curso actualizado,Sección del curso creada,completo
0,1013,4,1,531,1,5,2,1,5,1,...,0,0,0,0,0,0,0,0,0,1
1,1035,2,0,39,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1063,0,0,87,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1067,2,0,104,0,0,0,1,3,2,...,0,0,0,0,0,0,0,0,0,1
4,1082,4,0,83,0,0,0,1,2,2,...,0,0,0,0,0,0,0,0,0,0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 46 columns):
 #   Column                                           Non-Null Count  Dtype
---  ------                                           --------------  -----
 0   Nombre.completo.del.usuario                      151 non-null    int64
 1   Algún contenido ha sido publicado.               151 non-null    int64
 2   Comentario creado                                151 non-null    int64
 3   Curso visto                                      151 non-null    int64
 4   Entrega creada.                                  151 non-null    int64
 5   Finalización de actividad de curso actualizada   151 non-null    int64
 6   Formulario de entrega visto.                     151 non-null    int64
 7   Ha comenzado el intento                          151 non-null    int64
 8   Intento de cuestionario visualizado              151 non-null    int64
 9   Intento del cuestionario revisado                151 n

In [9]:
data.describe()

,Nombre.completo.del.usuario,Algún contenido ha sido publicado.,Comentario creado,Curso visto,Entrega creada.,Finalización de actividad de curso actualizada,Formulario de entrega visto.,Ha comenzado el intento,Intento de cuestionario visualizado,Intento del cuestionario revisado,...,Informe usuario del curso visto,Suscripción activada,Curso actualizado,Grupo creado,Elemento de calificación creado,Curso creado,Instancia de inscripción creada,Sección de curso actualizado,Sección del curso creada,completo
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,...,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,5104.920530,1.059603,0.006623,142.635762,0.033113,2.874172,0.092715,0.483444,1.933775,0.523179,...,0.006623,0.026490,0.033113,0.013245,0.006623,0.006623,0.019868,0.006623,0.033113,0.476821
std,2581.667873,2.355792,0.081379,124.013896,0.179526,4.587381,0.405808,0.587132,3.535852,0.855059,...,0.081379,0.161122,0.268756,0.162758,0.081379,0.081379,0.244137,0.081379,0.406894,0.501125
min,1013.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2870.000000,0.000000,0.000000,57.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5092.000000,0.000000,0.000000,109.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7208.500000,1.000000,0.000000,195.500000,0.000000,4.000000,0.000000,1.000000,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9999.000000,16.000000,1.000000,845.000000,1.000000,19.000000,3.000000,2.000000,23.000000,4.000000,...,1.000000,1.000000,3.000000,2.000000,1.000000,1.000000,3.000000,1.000000,5.000000,1.000000


### Se realizan los graficos para un analisis inicial de limpieza de datos